In [15]:
# Initial imports
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [16]:
# Loading data
file_path = Path("Resources/AHM_cleaned.csv")
AHM_cleaned_df = pd.read_csv(file_path)
AHM_cleaned_df.head()

,zpid,zipcode,latitude,longitude,latestPrice,livingAreaSqFt,latest_saledate,latest_saledate.1
0,111373431,78660,30.430632,-97.663078,305000.0,2601.0,2019-09-02,2019-09-02
1,120900430,78660,30.432673,-97.661697,295000.0,1768.0,2020-10-13,2020-10-13
2,2084491383,78660,30.409748,-97.639771,256125.0,1478.0,2019-07-31,2019-07-31
3,120901374,78660,30.432112,-97.661659,240000.0,1678.0,2018-08-08,2018-08-08
4,60134862,78660,30.437368,-97.656860,239900.0,2132.0,2018-10-31,2018-10-31


In [17]:
# Define features set (delete the target column in this case "bad") 
# X is the input
X = AHM_cleaned_df.copy()
X = X.drop("latestPrice", axis=1)
X.head()

,zpid,zipcode,latitude,longitude,livingAreaSqFt,latest_saledate,latest_saledate.1
0,111373431,78660,30.430632,-97.663078,2601.0,2019-09-02,2019-09-02
1,120900430,78660,30.432673,-97.661697,1768.0,2020-10-13,2020-10-13
2,2084491383,78660,30.409748,-97.639771,1478.0,2019-07-31,2019-07-31
3,120901374,78660,30.432112,-97.661659,1678.0,2018-08-08,2018-08-08
4,60134862,78660,30.437368,-97.656860,2132.0,2018-10-31,2018-10-31


In [18]:
# Define target vector
y = AHM_cleaned_df["latestPrice"].values.reshape(-1, 1)
y[:5]

array([[305000.],
       [295000.],
       [256125.],
       [240000.],
       [239900.]])

In [19]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [20]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(11378, 7)
(3793, 7)
(11378, 1)
(3793, 1)


In [21]:
# Splitting into Train and Test sets into an 80/20 split.
X_train2, X_test2, y_train2, y_test2 = train_test_split(X, y, random_state=78, train_size=0.80)

In [22]:
print(X_train2.shape)
print(X_test2.shape)
print(y_train2.shape)
print(y_test2.shape)

(12136, 7)
(3035, 7)
(12136, 1)
(3035, 1)


In [23]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [24]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

ValueError: could not convert string to float: '2018-12-21'